# Dewan Lab Neuron Pseudopopulation Analysis
## Import Dependencies

In [ ]:
import itertools
import os
os.environ['ISX'] = '0'

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from sklearn import metrics

from dewan_calcium import classifiers
from dewan_calcium import stats as dewan_stats
from dewan_calcium.helpers import IO
from dewan_calcium.helpers.project_folder import ProjectFolder

pd.options.mode.copy_on_write = "warn"

print('Finished importing required libraries!')

## Load Data from Project Folder

In [ ]:
# Create Project Folder to Gather and Hold all the File Paths
project_folder = ProjectFolder('ODOR', combined=True)

In [ ]:
# If this is the first time the project folder has been created,
# move the files to the appropriate directories and then run this cell, otherwise skip this cell
project_folder.get_data()

## Configs


In [ ]:
ANALYSIS_VARIABLE = 'odor' # Either 'odor' or 'block'

CELL_CLASS = 'vglut' # vglut or vgat
IDENTITY_EXP = True  # False if Concentration Experiment

WINDOW = 2 # Window size for the moving-window SVM decoder; set to None for no window and to consider all data at once
NUM_SVM_SPLITS = 20  # Number of random test-train splits to run and average per SVM run

# Values used in the combine-data.py standalone script to define the sizes of the different data periods
BASELINE_FRAMES = 20
ODOR_FRAMES = 20
POST_FRAMES = 20

SHOW_FIGURES = False

## ============================ CONSTANTS ============================ ##
VARS = ['odor', 'block']
CLASSES = ['vglut', 'vgat', 'oxtr']


ID_ORDER = ['4ATE', '5ATE', '6ATE', '7ATE', '4AL', '5AL', '6AL', '7AL', '4AMINE', '5AMINE', '6AMINE', '7AMINE', '4OL', '5OL', '6OL', '7OL', '4ONE', '5ONE', '6ONE', '7ONE']
CONC_ORDER = ['1ATE', '10ATE', '100ATE', '1000ATE', '1AL', '10AL', '100AL', '1000AL', '1AMINE', '10AMINE', '100AMINE', '1000AMINE', '1OL', '10OL', '100OL', '1000OL', '1ONE', '10ONE', '100ONE', '1000ONE']

LEVEL_MAP = {
    'odor': 2, # Level = 2 if we want to only look at odors; drops block labels
    'block': 1 # level = 1 if we want to only look at blocks; drops odor labels
}

CM_WINDOWS = {
   'Baseline': (0,20),
   'Odor_Period': (20,40),
   'Latent_Period': (40,60),
   'Odor_and_Latent': (20,60)
}

## VALIDATE INPUTS

IO.verify_input('ANALYSIS_VARIABLE', ANALYSIS_VARIABLE, [str], allowed_values=VARS)
IO.verify_input('CELL_CLASS', CELL_CLASS, [str], allowed_values=CLASSES)
IO.verify_input('IDENTITY_EXP', IDENTITY_EXP, [bool])
IO.verify_input('WINDOW', WINDOW, [int], allowed_range=(1, 20))
IO.verify_input('NUM_SVM_SPLITS', NUM_SVM_SPLITS, [int], allowed_range=(1, 100), inclusive=True)


## Check that the data file exists

In [ ]:
data_file = []
sig_table = []
if project_folder.raw_data_dir.combined_data_path:
    if CELL_CLASS.lower() in str(project_folder.raw_data_dir.combined_data_path).lower():
        data_file = project_folder.raw_data_dir.combined_data_path[0]
else:
    raise FileExistsError(f'No data file with class {CELL_CLASS} exists!')

if project_folder.raw_data_dir.combined_sig_table_path:
    if CELL_CLASS.lower() in str(project_folder.raw_data_dir.combined_sig_table_path).lower():
        sig_table = project_folder.raw_data_dir.combined_sig_table_path[0]
else:
    raise FileExistsError(f'No significance table with class {CELL_CLASS} exists!')

## Load and Z-Score Data

In [ ]:
combined_data = pd.read_pickle(data_file, compression={'method': 'xz'})

In [ ]:
## Z-Score Fluorescence Data
## Odors are presented to the animals in 3 blocks of seven odors each. During block 1, the average fluorescence compared to blocks 2 and 3 are elevated. Simultaneously, each cell
## has a variable fluorescence that needs to be normalized. To eliminate these effects, cells are internally z-scored per block. Data is segmented by block, and then each cell is independently z-scored.
## This allows the CellX-Block1 trials to be normalized separately from the CellX-Block2 trials which are separate from the CellX-Block3 trials.

def z_score_cell_data(df):
    cell_name = df.index.get_level_values(0).unique()
    df = df.loc[cell_name]
    return df.apply(stats.zscore)

def z_score_block_data(block_df):
    return block_df.groupby(level=0, group_keys=False).apply(z_score_cell_data)
    # Block_df contains the trials per block, each cell (level 0), now should be zscored against itself

z_scored_combined_data = combined_data.T.groupby(level=2, group_keys=False).apply(z_score_block_data).T
# Transform our dataframe to put the cell/odor/block as the index, group by level=2 (experiment block), apply stats.zscore to each group, transform back

cells = np.unique(combined_data.columns.get_level_values(0).values)
original_columns = z_scored_combined_data.columns

z_scored_combined_data.columns = z_scored_combined_data.columns.droplevel(level=LEVEL_MAP[ANALYSIS_VARIABLE])

In [ ]:
# Run this cell to reset the dataframe to its original configuration
# z_scored_combined_data.columns = original_columns

## SVM Classifier


### Sliding Window Decoding

In [ ]:
# Create Output Directories
svm_output_dir = project_folder.analysis_dir.output_dir.subdir('SVM')

if WINDOW:
    svm_output_dir = svm_output_dir.joinpath(f'Window-{WINDOW}')

svm_output_dir = svm_output_dir.joinpath(ANALYSIS_VARIABLE)

if not svm_output_dir.exists():
    svm_output_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
### Sliding Window Decoding
mean_svm_scores, splits_v_repeat_df, all_confusion_mats, (true_labels, pred_labels) = classifiers.sliding_window_ensemble_decoding(z_scored_combined_data, window_size=WINDOW, num_splits=NUM_SVM_SPLITS)

mean_score_df = pd.DataFrame(mean_svm_scores, np.arange(len(mean_svm_scores)))
mean_score_df.insert(0, column='num_cells',value=len(cells))

if WINDOW:
    mean_score_df.insert(0, column='window_size', value=WINDOW)

IO.save_SVM_output(svm_output_dir, mean_score_df, mean_svm_scores, splits_v_repeat_df, all_confusion_mats, true_labels, pred_labels)

### Shuffled Sliding Window Decoding
shuffled_data = classifiers.shuffle_data(z_scored_combined_data)

shuffled_mean_svm_scores, shuffled_splits_v_repeat_df, shuffled_all_confusion_mats, (shuffled_true_labels, shuffled_pred_labels) = classifiers.sliding_window_ensemble_decoding(shuffled_data, window_size=WINDOW, num_splits=NUM_SVM_SPLITS)

shuffled_mean_score_df = pd.DataFrame(shuffled_mean_svm_scores, np.arange(len(shuffled_mean_svm_scores)))
shuffled_mean_score_df.insert(0, column='num_cells',value=len(cells))

if WINDOW:
    shuffled_mean_score_df.insert(0, column='window_size', value=WINDOW)

IO.save_SVM_output(svm_output_dir, shuffled_mean_score_df, shuffled_mean_svm_scores, shuffled_splits_v_repeat_df, shuffled_all_confusion_mats, shuffled_true_labels, shuffled_pred_labels, shuffle=True)

## Checkpoint: Load SVM Output

In [ ]:
input_dir = project_folder.analysis_dir.output_dir.subdir('SVM')
if WINDOW:
    input_dir = input_dir.joinpath(f'Window-{WINDOW}')
input_dir = input_dir.joinpath(ANALYSIS_VARIABLE)

mean_scores_path = input_dir.joinpath('mean_svm_scores.pickle')
mean_svm_scores = pd.read_pickle(mean_scores_path)
splits_path = input_dir.joinpath('splits_v_repeat_df.pickle')
splits_v_repeat_df = pd.read_pickle(splits_path)
all_confusion_mat_path = input_dir.joinpath('all_confusion_mat.pickle')
all_confusion_mats = pd.read_pickle(all_confusion_mat_path)
labels_path = input_dir.joinpath('labels.pickle')
(true_labels, pred_labels) = pd.read_pickle(labels_path)

print('Successfully loaded SVM output!')
shuffle_input_dir = input_dir.joinpath('Shuffle')

shuffled_mean_scores_path = shuffle_input_dir.joinpath('mean_svm_scores.pickle')
shuffled_mean_svm_scores = pd.read_pickle(shuffled_mean_scores_path)
shuffled_splits_path = shuffle_input_dir.joinpath('splits_v_repeat_df.pickle')
shuffled_splits_v_repeat_df = pd.read_pickle(splits_path)
shuffled_all_confusion_mat_path = shuffle_input_dir.joinpath('all_confusion_mat.pickle')
shuffled_all_confusion_mats = pd.read_pickle(shuffled_all_confusion_mat_path)
shuffled_labels_path = shuffle_input_dir.joinpath('labels.pickle')
(shuffled_true_labels, shuffled_pred_labels) = pd.read_pickle(shuffled_labels_path)
print('Successfully loaded shuffled SVM output!')

In [ ]:
## Load/Create output directories

svm_output_dir = project_folder.analysis_dir.output_dir.subdir('SVM')
svm_fig_dir = project_folder.analysis_dir.figures_dir.subdir('SVM')

if WINDOW:
    svm_output_dir = svm_output_dir.joinpath(f'Window-{WINDOW}')
    svm_fig_dir = svm_fig_dir.joinpath(f'Window-{WINDOW}')

svm_output_dir = svm_output_dir.joinpath(ANALYSIS_VARIABLE)
svm_fig_dir = svm_fig_dir.joinpath(ANALYSIS_VARIABLE)

if not svm_output_dir.exists():
    svm_output_dir.mkdir(parents=True, exist_ok=True)
if not svm_fig_dir.exists():
    svm_fig_dir.mkdir(parents=True, exist_ok=True)

cm_data_save_dir = svm_output_dir.joinpath('CM')
cm_figure_save_dir = svm_fig_dir.joinpath('CM')

if not cm_data_save_dir.exists():
    cm_data_save_dir.mkdir(parents=True, exist_ok=True)
if not cm_figure_save_dir.exists():
    cm_figure_save_dir.mkdir(parents=True, exist_ok=True)

## Average individual confusion matrices

In [ ]:
windows = list(all_confusion_mats.keys())
window_averaged_cms = {}

for window in windows:
    window_cm = all_confusion_mats[window]
    avg_cm = np.mean(window_cm, axis=0)
    window_averaged_cms[window] = avg_cm

# Plot and Save average confusion matrices per window

for cm_window in CM_WINDOWS:
    odor_cm = []
    start_idx, end_idx = CM_WINDOWS[cm_window]
    for window in windows:
        if window[0] >= start_idx and window[1] <= end_idx:
            odor_cm.append(window_averaged_cms[window])

    average_odor_cm = np.mean(odor_cm, axis=0)

    if ANALYSIS_VARIABLE == 'odor':
        if IDENTITY_EXP:
            _columns = ID_ORDER
        else:
            _columns = CONC_ORDER
    else:
        _columns = [1, 2, 3]

    average_odor_cm_df = pd.DataFrame(average_odor_cm, columns=_columns, index=_columns)

    title_text = ' '.join(cm_window.split('_'))
    title_with_index = f'{title_text}({start_idx}-{end_idx})'
    df_save_path = cm_data_save_dir.joinpath(f'{title_with_index}.xlsx')
    fig_save_path = cm_figure_save_dir.joinpath(f'{title_with_index}.pdf')

    avg_cm = metrics.ConfusionMatrixDisplay(average_odor_cm, display_labels=_columns)
    avg_cm.plot(include_values=False)
    avg_cm.ax_.set_title(title_with_index)
    avg_cm.ax_.tick_params(axis='x', labelrotation=90)
    plt.tight_layout()
    avg_cm.figure_.savefig(fig_save_path, dpi=900)
    average_odor_cm_df.to_excel(df_save_path, index=True)

    if not SHOW_FIGURES:
        plt.close()


### Plot SVM Performance

In [ ]:
# Unpack Means
mean_performance = [mean_svm_scores[key] for key in mean_svm_scores]
shuffle_mean_performance = [shuffled_mean_svm_scores[key] for key in shuffled_mean_svm_scores]

# Calculate Confidence Intervals
sqrt = np.sqrt(splits_v_repeat_df.shape[0])

std_devs = splits_v_repeat_df.T.std(axis=1)
shuffl_std_devs = shuffled_splits_v_repeat_df.T.std(axis=1)

CI = 2.576 * (std_devs/sqrt)
shuffl_CI = 2.576 * (shuffl_std_devs/sqrt)

CI_min = np.subtract(mean_performance, CI)
CI_max = np.add(mean_performance, CI)
shuffle_CI_min = np.subtract(shuffle_mean_performance, shuffl_CI)
shuffle_CI_max = np.add(shuffle_mean_performance, shuffl_CI)

output_path = svm_output_dir.joinpath('SVM_Performance_Stats.xlsx')
_index = [(int(item[0]), int(item[1])) for item in list(mean_svm_scores.keys())]
svm_df = pd.DataFrame(np.vstack([mean_performance, CI_min, CI_max, shuffle_mean_performance, shuffle_CI_min, shuffle_CI_max]).T, index=_index, columns=['Mean SVM Performance', '99% CI Min', '99% CI Max', 'Shuffled SVM Performance', 'Shuffled 99% CI Min', 'Shuffled 99% CI Max'])
svm_df.to_excel(output_path)

In [ ]:
# Plot SVM performance vs. Shuffled Data
fig, ax = plt.subplots()
x_vals = np.linspace(-2, 3.5, len(mean_performance), endpoint=True) + 0.25
ax.plot(x_vals, mean_performance, color='#04BBC9', linewidth=3)
ax.plot(x_vals, shuffle_mean_performance, color='#C500FF', linewidth=1)
ax.fill_between(x_vals, CI_min, CI_max, alpha=0.5, color='red')
ax.fill_between(x_vals, shuffle_CI_min, shuffle_CI_max, alpha=0.5, color='green')

x_vals = np.linspace(-2, 4, 13)
plt.xticks(x_vals)
ax.vlines(x=0, ymin=-1, ymax=1, color='r')
ax.set_ylim([-0.05, 1])
ax.set_xlim([-2.1, 4.1])

if IDENTITY_EXP:
    _exp_type = 'ID'
else:
    _exp_type = 'CONC'

plt.suptitle(f'{CELL_CLASS} {_exp_type} SVM Classifier n={len(cells)}', fontsize=18, fontweight='bold')
ax.set_ylabel('Classifier Performance', fontsize=12)
ax.set_xlabel('Time Relative to Odor Onset (s)', fontsize=12)
plt.tight_layout()
plt.savefig(svm_fig_dir.joinpath(f'{CELL_CLASS}_{_exp_type}_Classifier_Odor.pdf'), dpi=600)

if not SHOW_FIGURES:
    plt.close()

In [ ]:
# If the SVM was run by block, plot the mean dF/F for each window
if ANALYSIS_VARIABLE == 'block':
    grouped_by_block = z_scored_combined_data.T.groupby(level=1)
    windows = list(mean_svm_scores.keys())

    block_windowed_means = {}

    for name, block in grouped_by_block:
        block_means = []
        for window in windows:
            start_idx, end_idx = window
            data = block.iloc[:, start_idx:end_idx]
            block_means.append(data.median().mean())

        block_windowed_means[name] = block_means

    block_1_means = block_windowed_means[1]
    block_2_means = block_windowed_means[2]
    block_3_means = block_windowed_means[3]
    _y_min = np.min([block_1_means, block_2_means, block_3_means]) * 1.1
    _y_max = np.max([block_1_means, block_2_means, block_3_means]) * .9

    x_vals = np.linspace(-2, 3.5, len(block_1_means), endpoint=True,) + 0.25
    fig,ax = plt.subplots()

    ax.plot(x_vals, block_1_means, color='red', linewidth=3, label='Block 1')
    ax.plot(x_vals, block_2_means, color='cyan', linewidth=3, label='Block 2')
    ax.plot(x_vals, block_3_means, color='green', linewidth=3, label='Block 3')
    ax.legend()
    ax.vlines(x=0, ymin=_y_min, ymax=_y_max, color='m')
    ax.set_ylim([_y_min, _y_max])

    x_vals = np.linspace(-2, 4, 13)
    plt.xticks(x_vals)
    ax.vlines(x=0, ymin=_y_min, ymax=_y_max, color='r')
    ax.set_xlim([-2.1, 4.1])

    plt.suptitle(f'{CELL_CLASS} {_exp_type} Average dF/F per Block n={len(cells)}', fontsize=18, fontweight='bold')
    ax.set_ylabel('average dF/F', fontsize=12)
    ax.set_xlabel('Time Relative to Odor Onset (s)', fontsize=12)
    plt.tight_layout()
    plt.savefig(svm_fig_dir.joinpath(f'{CELL_CLASS}_{_exp_type}_block_meandff.pdf'), dpi=600)

    if not SHOW_FIGURES:
        plt.close()

## Population and Lifetime Sparseness


### Calculate Cell-Odor Means

In [ ]:
# Pop Sparseness -> per odor
# Lifetime Sparseness -> per cell
z_scored_combined_data.columns = original_columns

transposed_data = z_scored_combined_data.T

cell_medians = {}
cell_means = {}
nonzero_cell_medians = {}

cells = transposed_data.groupby('Cells')
cell_odor_orders = []

for cell_name, cell_df in cells:
    medians = []
    means = []
    nonzero_medians = []
    odor_order = []
    odors = cell_df.groupby('Trials')

    for name, odor_data in odors:
        baseline_mean = odor_data.iloc[:, :BASELINE_FRAMES].mean(axis=1) # baseline means for each trial
        odor_trial_means = odor_data.iloc[:, BASELINE_FRAMES: BASELINE_FRAMES + ODOR_FRAMES].mean(axis=1) # odor evoked means for each trial
        diff = odor_trial_means.subtract(baseline_mean, axis=0)  # subtract the baseline from odor activity
        _mean = diff.mean()
        _median = diff.median()

        nonzero_medians.append(_median)

        # What happens if you zero the values BEFORE taking the mean?
        if _mean < 0:
            _mean = 0
        if _median < 0:
            _median = 0

        means.append(_mean)
        medians.append(_median)
        odor_order.append(name)

    cell_means[cell_name] = means
    cell_medians[cell_name] = medians
    nonzero_cell_medians[cell_name] = nonzero_medians
    cell_odor_orders.append(odor_order)

odors = cell_odor_orders[0]

cell_means = pd.DataFrame(cell_means, index=odors)
cell_medians = pd.DataFrame(cell_medians, index=odors)
nonzero_cell_medians = pd.DataFrame(nonzero_cell_medians, index=odors)

## Population Sparseness

In [ ]:
pop_sparseness_values_means = {}
pop_sparseness_values_medians = {}

for odor_name, odor_data in cell_means.iterrows():
    num_cells = odor_data.shape[0]
    odor_data = odor_data.values
    sparseness_value = dewan_stats.sparseness(num_cells, odor_data)
    pop_sparseness_values_means[odor_name] = sparseness_value

for odor_name, odor_data in cell_medians.iterrows():
    num_cells = odor_data.shape[0]
    odor_data = odor_data.values
    sparseness_value = dewan_stats.sparseness(num_cells, odor_data)
    pop_sparseness_values_medians[odor_name] = sparseness_value

pop_sparseness_values_means = pd.DataFrame(pop_sparseness_values_means, index=[0])
pop_sparseness_values_medians = pd.DataFrame(pop_sparseness_values_medians, index=[0])

## Lifetime Sparseness

In [ ]:
lifetime_sparseness_values_means = {}
lifetime_sparseness_values_medians = {}

for cell_name, cell_data in cell_means.items():
    num_odors = cell_data.shape[0]
    cell_data = cell_data.values
    sparseness_values = dewan_stats.sparseness(num_odors, cell_data)
    lifetime_sparseness_values_means[cell_name] = sparseness_values

for cell_name, cell_data in cell_medians.items():
    num_odors = cell_data.shape[0]
    cell_data = cell_data.values
    sparseness_values = dewan_stats.sparseness(num_odors, cell_data)
    lifetime_sparseness_values_medians[cell_name] = sparseness_values

lifetime_sparseness_values_means = pd.DataFrame(lifetime_sparseness_values_means, index=[0])
lifetime_sparseness_values_medians = pd.DataFrame(lifetime_sparseness_values_medians, index=[0])

## Write to File

In [ ]:
sparseness_path = project_folder.analysis_dir.output_dir.path.joinpath('sparseness.xlsx')

with pd.ExcelWriter(sparseness_path, engine='xlsxwriter') as writer:
    pop_sparseness_values_medians.to_excel(writer, sheet_name='Population Sparseness (Medians)')
    pop_sparseness_values_means.to_excel(writer, sheet_name='Population Sparseness (Means)')
    lifetime_sparseness_values_medians.to_excel(writer, sheet_name='Lifetime Sparseness (Medians)', index=[0])
    lifetime_sparseness_values_means.to_excel(writer, sheet_name='Lifetime Sparseness (Means)', index=[0])
    cell_means.to_excel(writer, sheet_name='Cell Means (Zeroed)')
    cell_medians.to_excel(writer, sheet_name='Cell Medians (Zeroed)')

## Correlations

In [ ]:
odors = transposed_data.index.get_level_values(1).unique()
odors = np.sort(odors)
perms = list(itertools.permutations(odors, 2))

correlations = pd.DataFrame(dtype=float, index=odors, columns=odors)  # Explicitly set dtype to float

for odor1, odor2 in perms:
    odor1_means = cell_means.loc[odor1]
    odor2_means = cell_means.loc[odor2]

    pearson_result = stats.pearsonr(odor1_means, odor2_means)

    correlations.loc[odor1, odor2] = pearson_result.statistic

correlations = correlations.fillna(1.0)
correlations = 1 - correlations

if IDENTITY_EXP:
    sorted_correlations = correlations.loc[ID_ORDER, ID_ORDER]
else:
    sorted_correlations = correlations.loc[CONC_ORDER, CONC_ORDER]

correlation_path = project_folder.analysis_dir.output_dir.path.joinpath('correlations.xlsx')
sorted_correlations.to_excel(correlation_path)

## Reorganize Cell Significance Matrix

In [ ]:
## Sorting Rules
"""
1) Non-responsive - zeros only
2) Excitatory on responses - 2s only (sort by the most 2s)
3) Excitatory off responses - 4s only (sort by the most 4s)
4) Excitatory combo responses- 2s and 4s (sort by the most 2s+4s)
5) Inhibitory on responses - 1s only (sort by the most 1s)
6) Inhibitory off responses - 3s only (sort by the most 3s)
7) Inhibitory combo responses (1s and 3s sort by the most 1s+3s)
8) Combo responses (any combination of 1/3 and 2/4 sort by the most responses)
9) Buzzer - any cell that responses to the buzzer (sort by the total number of responses of any number)
10) MO - any cell that responses to MO (sort by the total number of responses of any number)
"""

In [ ]:
def sort_by_response_number(df: pd.DataFrame):
    order = df.T.ne(0).sum().sort_values(ascending=False).index
    sorted_df = df.loc[order]
    return sorted_df

def get_and_sort_cells(odor_responsive_cells, IDs):
    ID_mask = np.all(odor_responsive_cells.isin(IDs), axis=1)
    ID_cells = odor_responsive_cells.loc[ID_mask]
    sorted_ID_cells = sort_by_response_number(ID_cells)
    return sorted_ID_cells

In [ ]:
combined_sig_table = pd.read_excel(sig_table, index_col=0)

nonresponsive_cells_mask = (combined_sig_table.sum(axis=1) == 0)
nonresponsive_cells = combined_sig_table.loc[nonresponsive_cells_mask]

responsive_cells_mask = np.logical_not(nonresponsive_cells_mask)
responsive_cells = combined_sig_table.loc[responsive_cells_mask]

buzzer_mask = (responsive_cells['Buzzer'] != 0)
buzzer_cells = responsive_cells.loc[buzzer_mask]
buzzer_cells = sort_by_response_number(buzzer_cells)

non_buzzer_mask = np.logical_not(buzzer_mask)
non_buzzer_cells = responsive_cells.loc[non_buzzer_mask]

MO_mask = (non_buzzer_cells['MO'] != 0)
MO_cells = non_buzzer_cells.loc[MO_mask]
MO_cells = sort_by_response_number(MO_cells)

non_MO_mask = np.logical_not(MO_mask)
odor_responsive_cells = non_buzzer_cells.loc[non_MO_mask]

excitatory_on_cells = get_and_sort_cells(odor_responsive_cells, [0, 2])   # 0 and 2 ONLY
odor_responsive_cells = odor_responsive_cells.drop(excitatory_on_cells.index)

excitatory_off_cells = get_and_sort_cells(odor_responsive_cells, [0, 4])   # 0 and 4 ONLY
odor_responsive_cells = odor_responsive_cells.drop(excitatory_off_cells.index)

excitatory_combo_cells = get_and_sort_cells(odor_responsive_cells, [0, 2, 4]) # 0, 2, AND 4 ONLY
odor_responsive_cells = odor_responsive_cells.drop(excitatory_combo_cells.index)

inhibitory_on_cells = get_and_sort_cells(odor_responsive_cells, [0, 1])  # 0 and 1 ONLY
odor_responsive_cells = odor_responsive_cells.drop(inhibitory_on_cells.index)
inhibitory_off_cells = get_and_sort_cells(odor_responsive_cells, [0, 3]) # 0 and 3 ONLY
odor_responsive_cells = odor_responsive_cells.drop(inhibitory_off_cells.index)
inhibitory_combo_cells = get_and_sort_cells(odor_responsive_cells, [0, 1, 3]) # 0, 1 AND 3 ONLY
odor_responsive_cells = odor_responsive_cells.drop(inhibitory_combo_cells.index)

any_combo_cells = sort_by_response_number(odor_responsive_cells)

sorted_dataframe = pd.concat([nonresponsive_cells, excitatory_on_cells, excitatory_off_cells, excitatory_combo_cells, inhibitory_on_cells, inhibitory_off_cells, inhibitory_combo_cells, any_combo_cells, buzzer_cells, MO_cells])

if IDENTITY_EXP:
    _ID_ORDER = np.hstack([ID_ORDER, ['Buzzer', 'MO']])
    sorted_by_odor = sorted_dataframe[_ID_ORDER]
else:
    _CONC_ORDER = np.hstack([CONC_ORDER, ['Buzzer', 'MO']])
    sorted_by_odor = sorted_dataframe[_CONC_ORDER]

sorted_sig_table_path = project_folder.analysis_dir.output_dir.path.joinpath('sorted_significance_table.xlsx')
sorted_by_odor.to_excel(sorted_sig_table_path)